In [1]:
import sys
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/database_functions/')
from database import Database
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/modules/')
from utils import Format_data
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

db = Database()
fd = Format_data()


2025-10-23 09:31:39,233 - INFO - Cliente Oracle inicializado com sucesso usando o caminho: H:\Tecnologia\EQUIPE - DADOS\dev_env\000_base\instantclient_19_21
2025-10-23 09:31:42,599 - INFO - Banco de dados conectado com sucesso!


In [2]:

# DADOS_CALENDARIO_BI

df_calendario = db.extract_data("""
     SELECT * FROM DADOS_CALENDARIO_BI dc
WHERE dc.MES_AJUSTADO = '01/10/2025'
AND dc.DATA > '15/10/2025'
    """)




# CRIAR UMA BASE DE MAILING ÚNICA PARA ESSA CAMPANHA ESPECÍFICA
query_ortodontia  = db.extract_data("""

WITH BASE_ORTO AS (
 -- Etapa 1: Seleciona pacientes que fizeram atendimento de ortodontia há exatos 35 dias
    SELECT
        BI.COD_PACIENTE,
        P.NOME                                                                                         AS NOME_PACIENTE,
        REPLACE(REPLACE(P.FONE1, '(', ''), ')', '')                                                    AS TELEFONE,
        BI.PROCEDIMENTO,
        BI.DATA_ATENDIMENTO,
        BI.DT_PG,
        ROW_NUMBER() OVER ( PARTITION BY BI.COD_PACIENTE ORDER BY BI.DT_PG DESC, BI.DATA_ATENDIMENTO ) AS RN
    FROM
        AGE.BI_PAGAMENTOS BI
        INNER JOIN AGE.PACIENTE P
        ON BI.COD_PACIENTE = P.COD_PACIENTE
    WHERE
        BI.GRUPO = 'ODONTOLOGIA'
        AND BI.PROCEDIMENTO LIKE '%APARELHO%'
        AND BI.PROCEDIMENTO NOT LIKE '%REMOCAO%'
        AND BI.PROCEDIMENTO NOT LIKE '%RETIRADA%'
        AND BI.SERVICO <> 'LAPROJAL'
        AND BI.DT_PG = TRUNC(CURRENT_DATE) - 35
)
 -- Etapa 2: Seleciona os pacientes que não reagendaram nos últimos 35 dias
-- e mostra se estão agendados para ainda este mês (usando AGENDA_HORARIO)
SELECT
    TO_CHAR(ORT.DT_PG, 'DD/MM/YYYY') AS ULTIMO_ATEND,
    ORT.COD_PACIENTE,
    ORT.NOME_PACIENTE,
    ORT.TELEFONE,
    ORT.PROCEDIMENTO,
    'ORTODONTIA'                     AS CAMPANHA, 
    -- Próxima data de agendamento futura dentro do mês atual (se existir)
    (
        SELECT
            TO_CHAR(MIN(A.DATA_AGENDA),
            'DD/MM/YYYY')
        FROM
            AGENDA_HORARIO A
        WHERE
            A.COD_PACIENTE = ORT.COD_PACIENTE
            AND A.DATA_AGENDA > CURRENT_DATE
            AND EXTRACT(MONTH FROM A.DATA_AGENDA) = EXTRACT(MONTH FROM CURRENT_DATE)
            AND EXTRACT(YEAR FROM A.DATA_AGENDA) = EXTRACT(YEAR FROM CURRENT_DATE)
    ) AS AGENDAMENTO_MES_ATUAL
FROM
    BASE_ORTO      ORT
WHERE
    ORT.RN = 1
    AND NOT EXISTS (
        SELECT
            1
        FROM
            AGENDA_HORARIO A
        WHERE
            A.COD_PACIENTE = ORT.COD_PACIENTE
            AND A.DATA_AGENDA BETWEEN (CURRENT_DATE - 35) AND CURRENT_DATE
    )
ORDER BY
    ORT.DT_PG ASC
    """)
#--------------------------------------------------------------------------------------------------------------------


2025-10-23 09:31:53,236 - INFO - Dados coletados com sucesso!
2025-10-23 09:31:53,278 - INFO - Dados coletados com sucesso!


In [4]:
print(f"📦 Total de registros extraídos: {query_ortodontia.shape[0]}")

📦 Total de registros extraídos: 8


In [5]:
df_agendamentos = query_ortodontia

# 🧹 Filtra os casos onde AGENDAMENTO_MES_ATUAL está vazio ou nulo
df_sem_agendamento = df_agendamentos[
    df_agendamentos['AGENDAMENTO_MES_ATUAL'].isnull() |
    (df_agendamentos['AGENDAMENTO_MES_ATUAL'].astype(str).str.strip() == '')
]

df_base_maling = df_sem_agendamento.copy()


# 💾 Exibe ou salva o resultado
print(f"🔍 Total de pacientes sem agendamento este mês: {len(df_base_maling)}")
display(df_base_maling)

🔍 Total de pacientes sem agendamento este mês: 8


,ULTIMO_ATEND,COD_PACIENTE,NOME_PACIENTE,TELEFONE,PROCEDIMENTO,CAMPANHA,AGENDAMENTO_MES_ATUAL
0,18/09/2025,152771,NEUZA LOPES RODRIGUES,51991542727,423 MANUTENCAO DE APARELHO ASSINANTE - SERV,ORTODONTIA,None
1,18/09/2025,736755,CESAR LEONE DE SOUZA MARQUES,51981245920,422 MANUTENCAO DE APARELHO ORTODONTICO - SERV,ORTODONTIA,None
2,18/09/2025,1711474,JULY MARCELI COSTA DA SILVA,51996623238,423 MANUTENCAO DE APARELHO ASSINANTE - SERV,ORTODONTIA,None
3,18/09/2025,1872726,ARTHUR JUSTO DELFINO,51984939159,422 MANUTENCAO DE APARELHO ORTODONTICO - SERV,ORTODONTIA,None
4,18/09/2025,1903265,EDUARDO OLIVEIRA DE MATTOS BASTOS,51993014123,422 MANUTENCAO DE APARELHO ORTODONTICO - SERV,ORTODONTIA,None
5,18/09/2025,2011787,ISABELLE BEATRIZ LOZANO SAIFER DOS SANTOS,51998085788,422 MANUTENCAO DE APARELHO ORTODONTICO - SERV,ORTODONTIA,None
6,18/09/2025,2012258,LUCAS STUMPF ROCKSTROH,51999872426,422 MANUTENCAO DE APARELHO ORTODONTICO - SERV,ORTODONTIA,None
7,18/09/2025,2042679,LAIS DA ROCHA SANTA MARIA,51993228137,422 MANUTENCAO DE APARELHO ORTODONTICO - SERV,ORTODONTIA,None


In [6]:

# 🔹 1. Limpeza de telefone
df_base_maling['TELEFONE_LIMPO'] = df_base_maling['TELEFONE'].astype(str).str.replace(r'[^0-9]', '', regex=True)
df_base_maling['QTDE_DIGITOS'] = df_base_maling['TELEFONE_LIMPO'].str.len()

# 🔹 2. Filtra telefones válidos
df_filtrado = df_base_maling[
    (df_base_maling['QTDE_DIGITOS'] > 10) &
    (df_base_maling['TELEFONE_LIMPO'].notnull()) &
    (df_base_maling['TELEFONE_LIMPO'].str.contains(r'\d', na=False))
].copy()

# 🔹 3. Filtra campanha específica
df_campanha = df_filtrado[df_filtrado['CAMPANHA'] == 'ORTODONTIA'].copy()
df_campanha['CANAIS'] = 1
df_campanha['STATUS'] = 'Enviar'

# 🔹 4. Distribuição de datas úteis
data_inicio = pd.to_datetime(datetime.today().date())
df_calendario['DATA'] = pd.to_datetime(df_calendario['DATA'], errors='coerce')

dias_uteis_validos = df_calendario[
    (df_calendario['FERIADO_FOLGA'] != 'S') &
    (df_calendario['DIAS_TT_MES'] > 0) &
    (df_calendario['DATA'] >= data_inicio)
][['DATA', 'DIAS_TT_MES']].copy()

datas_expandidas = []
for _, row in dias_uteis_validos.iterrows():
    peso = row['DIAS_TT_MES']
    rep = int(peso * 2)
    datas_expandidas.extend([row['DATA']] * rep)

if len(datas_expandidas) == 0:
    raise ValueError("⚠️ Nenhum dia útil disponível após hoje.")

total = len(df_campanha)
registros_por_dia = max(1, total // len(datas_expandidas))

df_campanha['LIGAR_EM'] = [
    datas_expandidas[i // registros_por_dia] if i // registros_por_dia < len(datas_expandidas) else datas_expandidas[-1]
    for i in range(total)
]
df_campanha['LIGAR_EM'] = pd.to_datetime(df_campanha['LIGAR_EM']).dt.strftime('%Y-%m-%d')

# 🔹 5. Inserção em lote
nome_tabela = 'AGE.CAMP_ORTODONTIA'

sql = f"""
    INSERT INTO {nome_tabela} (
        SEQ_CAMPANHA,
        COD_PACIENTE,
        NOME_PACIENTE,
        FONE,
        CAMPANHA,
        STATUS,
        LIGAR_EM,
        CANAIS      
    ) VALUES (
        SEQ_CAMPANHA.nextVal,
        :1, :2, :3, :4, :5, TO_DATE(:6, 'YYYY-MM-DD'), :7
    )
"""

# 🔹 6. Prepara os dados como tuplas
dados = [
    (
        row['COD_PACIENTE'],
        str(row['NOME_PACIENTE']).replace("'", "''"),
        str(row['TELEFONE_LIMPO']).replace("'", "''"),
        str(row['CAMPANHA']).replace("'", "''"),
        row['STATUS'],
        row['LIGAR_EM'],
        row['CANAIS']
    )
    for _, row in df_campanha.iterrows()
]

# 🔹 7. Executa em lote
try:
    db.cursor.executemany(sql, dados)
    db.connection.commit()
    print(f"✅ {len(dados)} registros inseridos com sucesso na tabela {nome_tabela}.")
except Exception as e:
    print(f"❌ Erro ao inserir dados: {e}")


✅ 8 registros inseridos com sucesso na tabela AGE.CAMP_ORTODONTIA.
